# csv파일 DB로 저장

In [7]:
import pandas as pd
import pymysql

bmi = pd.read_csv("data/bmi.csv")
print(bmi.info())

height = bmi['height']
weight = bmi['weight']
label = bmi['label']

confing = {
    'host' : '127.0.0.1',
    'user' : 'root',
    'password' : '0000',
    'database' : 'work',
    'port' : 3306,
    'charset' : 'utf8',
    'use_unicode' : True}

try:
    conn = pymysql.connect(**confing)
    cursor = conn.cursor()

    cursor.execute("show tables")
    tables = cursor.fetchall()
    # print("################")
    # print(tables)

    sw = False
    for table in tables:
        if table[0] == 'bmi_tab':
            sw = True
    if not sw:
        print('테이블 없음')
        sql = """create table bmi_tab(
        height int not null,
        weight int not null,
        label varchar(15) not null
        )"""
        cursor.execute(sql)
    
    cursor.execute("select * from bmi_tab")
    rows = cursor.fetchall()

    if rows:
        for row in rows:
            print(f"{row[0]}    {row[1]}    {row[2]}")
            print('전체 레코드 수 : ', len(rows))
    else:
        print("100 레코드 추가")
        for i in range(100):
            h = height[i]
            w = weight[i]
            lab = label[i]
            if (0<h<250) and (0<w<250):
                cursor.execute(f"insert into bmi_tab values({h}, {w}, '{lab}')")
                conn.commit()
except Exception as e:
    print('db error : ', e)
finally:
    cursor.close()
    conn.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   height  20000 non-null  int64 
 1   weight  20000 non-null  int64 
 2   label   20000 non-null  object
dtypes: int64(2), object(1)
memory usage: 468.9+ KB
None
184    61    thin
전체 레코드 수 :  100
189    56    thin
전체 레코드 수 :  100
183    79    normal
전체 레코드 수 :  100
143    40    normal
전체 레코드 수 :  100
187    66    normal
전체 레코드 수 :  100
161    52    normal
전체 레코드 수 :  100
186    54    thin
전체 레코드 수 :  100
144    57    fat
전체 레코드 수 :  100
184    55    thin
전체 레코드 수 :  100
165    76    fat
전체 레코드 수 :  100
143    45    normal
전체 레코드 수 :  100
154    61    fat
전체 레코드 수 :  100
159    57    normal
전체 레코드 수 :  100
177    45    thin
전체 레코드 수 :  100
174    78    fat
전체 레코드 수 :  100
140    80    fat
전체 레코드 수 :  100
150    75    fat
전체 레코드 수 :  100
141    47    normal
전체 레코드 수 :  100
186    57    thin
전체 레코드 수 :  100
1

# json파일 DB로 저장

In [9]:
import json
import pandas as pd
import pymysql

file = open("data/labels.json", mode='r', encoding='utf-8')

lines = json.load(file)
print(type(lines))
print(len(lines))
print(type(lines[0]))

df = pd.DataFrame(lines)
print(df.info())
print(df.head())

confing = {
    'host' : '127.0.0.1',
    'user' : 'root',
    'password' : '0000',
    'database' : 'work',
    'port' : 3306,
    'charset' : 'utf8',
    'use_unicode' : True}

try:
    conn = pymysql.connect(**confing)
    cursor = conn.cursor()

    sql = """create table labels(
    id int not null,
    url varchar(150) not null,
    name varchar(50) not null,
    color varchar(50) not null,
    de char(5)
    )"""
    cursor.execute(sql)

    cursor.execute("select * from labels")
    rows = cursor.fetchall()
    if rows:
        print("labels 레코드 조회")
        for row in rows:
            print(row)
        print("전체 레코드 수 : ", len(rows))
    else:
        for i in range(30):
            uid = df.id[i]
            url = df.url[i]
            name = df.name[i]
            color = df.color[i]
            de = str(df.default[i])
            sql = f"insert into labels values({uid}, '{url}', '{name}', '{color}', '{de}')"
            cursor.execute(sql)
            conn.commit()
except Exception as e:
    print("db error : ", e)
finally:
    cursor.close()
    conn.close()

<class 'list'>
30
<class 'dict'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       30 non-null     int64 
 1   url      30 non-null     object
 2   name     30 non-null     object
 3   color    30 non-null     object
 4   default  30 non-null     bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 1.1+ KB
None
       id                                                url         name  \
0   76811  https://api.github.com/repos/pandas-dev/pandas...          Bug   
1   76812  https://api.github.com/repos/pandas-dev/pandas...  Enhancement   
2  127681  https://api.github.com/repos/pandas-dev/pandas...     Refactor   
3  129350  https://api.github.com/repos/pandas-dev/pandas...        Build   
4  134699  https://api.github.com/repos/pandas-dev/pandas...         Docs   

    color  default  
0  e10c02    False  
1  4E9A06    False  
2  FCE94F    Fal